## Quick demo on contrastiveRegressor for EF

In [8]:
import os
import pandas as pd
import numpy as np
from simulation_utils import generate_students_data
import matplotlib.pyplot as plt
from sklearn.model_selection import train_test_split

from contrastiveRegressor import contrastiveRegressor

In [9]:
# create two groups of students with different correlations

correlation_between_time_and_score = [0.65, 0.85]

df_students = pd.concat([
  generate_students_data(num_samples=500, rho=correlation_between_time_and_score[0]),
  generate_students_data(num_samples=50, rho=correlation_between_time_and_score[1])
])

df_students.head()

,noise_1,noise_2,timeSpent,score
0,87,595,245.338121,47.453477
1,48,166,303.653605,45.988062
2,15,433,146.250126,25.395551
3,81,532,396.698942,67.106258
4,8,543,293.656748,37.722824


In [10]:
# test and train
inputVars = df_students.columns.tolist()
responseVar = 'score'
inputVars.remove(responseVar)


X_train, X_test, y_train, y_test = \
  train_test_split(df_students[inputVars], df_students[responseVar].values, \
  test_size=0.1)

In [11]:
# Regressor
num_iterations = 200
learning_rate  = 0.08
depth = 10
num_neighbours = 4
validation_test_size = 0.20
feat_importance_keyword = 'feature_importances_'

In [ ]:
# set the engine to CatBoost
from catboost import CatBoostRegressor
cb_model = CatBoostRegressor(iterations=num_iterations, learning_rate=learning_rate,
depth=depth, loss_function='RMSE', cat_features=None, silent=False)

In [12]:
# Create the regressor
contrastiveReg = contrastiveRegressor(num_neighbours = num_neighbours, 
validation_test_size = validation_test_size)

# Set the regressor
contrastiveReg.set_regressor(cb_model, feat_importance_keyword, inputVars)
# fit the regressor
contrastiveReg.fit(X_train.values, y_train)
# eval results
contrastiveReg.predict_eval_test()
eval_results = contrastiveReg.get_results()

Preparing Training set...
Training set (1584, 6). Evaluation (396, 6)...done.
...Symmetrical Weights
MAE: 4.02
MSE: 25.52
RMSE: 5.05
meanError: -0.32
MAPE: 10.39
R2: 0.92
frc_error: 0.08
frc_bias: -0.01
frc_acc: 1.01
Var explained: 0.92


In [13]:
# Predict
contrastiveReg.predict(X_test.values)
student_predictions = contrastiveReg.get_results()

...Symmetrical Weights


In [14]:
idx_review = 10
contrastiveReg.get_contrastive_explanation(idx_review, X_test, y_test)

,timeSpent,noise_1,noise_2,y_train,delta_y_train,y_train_plus_delta,y_train_distances,y_train_weights,y_actual,y_forecast,y_weighted_forecast
var_importance,48.891078,26.175254,24.933668,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
neighbour_0,343.628792,79.000000,350.000000,62.774354,3.233408,66.007762,0.454717,2.199170,NaN,NaN,NaN
neighbour_1,323.281745,62.000000,383.000000,66.715928,0.254084,66.970012,0.509896,1.961184,NaN,NaN,NaN
neighbour_2,358.215830,76.000000,320.000000,60.157802,5.323084,65.480886,0.527643,1.895221,NaN,NaN,NaN
neighbour_3,318.013140,64.000000,291.000000,54.938646,4.403134,59.341780,0.567701,1.761492,NaN,NaN,NaN
367,331.306063,71.000000,355.000000,NaN,NaN,NaN,NaN,NaN,39.458568,64.45011,64.61933
